In [ ]:
from langchain_community.utilities import SLQDatabase
from pyprojroot import here
import warmings
warmings.filterwarnings("ignore")

In [ ]:
db_path = str(here('data')) + '/sqldb.db'
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [ ]:
print(db.dialect)
print(db.get_table_info())
db.run("SELECT * FROM api_data LIMIT 10")

In [ ]:
from dotenv import load_dotenv
import os
print("Environment Variables Loaded: ", load_dotenv())
print("test by a reading variable: ", os.getenv("API_KEY"))

In [ ]:
from openai import OpenAI
message = [
    {"role": "user", "content": str("You aren a helpful assistant.")},
    {"role": "user", "content": str("Hello!")}
]

client = AzureOpenAI(
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    messages=message
)

print(response.choices[0].message.content)

In [ ]:
from langchain.chat_models import AzureChatOpenAI
model_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
llm = AzureChatOpenAI(
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    azure_develoyment=model_name,
    model_name=model_name,
    temperature-0.0)



In [ ]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invole({"question": "How many records are there in the api_data table?"})
print(response)

In [ ]:
db.run(response)

In [ ]:
chain.get_prompt()[e].pretty_print()

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool

write_query = crete_sql_query_chain(llm,db)
execute_query = QuerySQLDatabaseTool(db=db)

chain = write_query | execute_query

chain.invoke({"question": "How many records are there in the api_data table?"})

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.chains import RunnablePassthrough

answer_prompt = PromptTemplate(
    """Given the following SQL query result, provide a concise answer to the question.
    
    Question: {question}
    SQL Result: {result}
    SQL Query: {query}
    Answer:""",
)

answer = answer_prompt.partial | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
        )
        | answer
)

chain.invoke({"question": "How many records are there in the api_data table?"}) 

In [ ]:
chain

In [ ]:
from langchain_community.agent_tookits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [ ]:
agent_executor.invoke (

{"inpur": "Which department has the largest expenditure?"}

)

In [ ]:
agent_executor.invoke({"input": "How many records are there in the api_data table?"})